## Load Libraries
Load needed libraries and inspect data. Small simulated dataset with 5 steps V1:V5 and a target column for classificaiton (0= failure, 1= success)

In [1]:
library(magrittr)
library(dplyr)
library(glue)
library(data.table)
library(ggplot2)
library(tfdatasets)
library(tensorflow)
library(keras)
tf$version$VERSION
## load data Channel Dataset-----------------------------------------------------
load(file = 'df_paths.rdata',verbose = T)
df_paths


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘glue’


The following object is masked from ‘package:dplyr’:

    collapse



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last




[1] "2.1.0"

Loading objects:
  df_paths
  train_paths
  test_paths


## Feature Specification
Using Specification to create tensors and embeddings for channels.

In [2]:
ft_spec_target <- df_paths %>%
      select(-customer_id,-path_no,-path) %>% 
      feature_spec(target ~ .) %>%
      step_categorical_column_with_vocabulary_list(starts_with("V"),vocabulary_list = list('channel_0', 'channel_1', 'channel_2', 'channel_3', 'channel_4', 'channel_5', 'channel_6', 'channel_7', 'channel_8')) %>%
      step_embedding_column(starts_with("V"), dimension = function(vocab_size) as.integer(sqrt(vocab_size) + 1)   ## dim=4
      ) %>%
  fit()

ft_spec_target$dense_features() 

$embedding_V1
EmbeddingColumn(categorical_column=VocabularyListCategoricalColumn(key='V1', vocabulary_list=('channel_0', 'channel_1', 'channel_2', 'channel_3', 'channel_4', 'channel_5', 'channel_6', 'channel_7', 'channel_8'), dtype=tf.string, default_value=-1, num_oov_buckets=0), dimension=4, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True)

$embedding_V2
EmbeddingColumn(categorical_column=VocabularyListCategoricalColumn(key='V2', vocabulary_list=('channel_0', 'channel_1', 'channel_2', 'channel_3', 'channel_4', 'channel_5', 'channel_6', 'channel_7', 'channel_8'), dtype=tf.string, default_value=-1, num_oov_buckets=0), dimension=4, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True)

$embedding_V3
EmbeddingColumn(categorical_column=VocabularyListCategoricalColumn(key='V3',

## Build and Train RNN 
Building an RNN for sequence modeling usually is out first approach. It works but has a few kinks that we will observe.

In [3]:
cat('Build model...\n')
 n_lstm = 7
batch_size = 32

rnn_in <- layer_input_from_dataset(df_paths %>% select(-target)) 

rnn_out <- 
  rnn_in %>%
  layer_dense_features(ft_spec_target$dense_features()) %>%
  layer_reshape(target_shape= list(5,4))  %>%
  bidirectional( layer_lstm(units = n_lstm, dropout = 0.2, recurrent_dropout = 0.2) ) %>%  
  layer_dense(units = 1, activation = 'sigmoid')

rnn <- keras_model(rnn_in, rnn_out)

rnn %>% 
  compile(
    loss = "binary_crossentropy", 
    optimizer = "adam",
    metrics = "accuracy"
  )

rnn %>% fit(x = df_paths %>% select(-target), y = df_paths$target, verbose=2,
            validation_split = 0.2, epoch=20 ,batch_size=batch_size )

Build model...


In [ ]:
rnn_scored = cbind( df_paths , rnn_pred = predict(rnn, df_paths) ) 

## Flip Channel
Alter channel in step 4 and observe the prediction scores swing. It more prominent when data space (possibilities) is large and observations not that many. 


In [7]:
rnn_scored[customer_id=='id99']

## alter Step 4 to channel-5 
df_altered1 = df_paths[customer_id=='id99', V4 := 'channel_5'][customer_id=='id99'] 

predict(rnn, df_altered1 )  ## Change in pred

## alter Step 4 to channel-0 (means not to anything)
df_altered2 = df_paths[customer_id=='id99', V4 := 'channel_0'][customer_id=='id99'] 
predict(rnn, df_altered2 )  ## Change in pred

customer_id,path_no,path,V1,V2,V3,V4,V5,target,rnn_pred.V1
<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
id99,1,channel_8->channel_1->channel_6->channel_1->channel_1->channel_1->channel_8,channel_8,channel_1,channel_6,channel_1,channel_1,0,0.4099931


0.9980208


0.4932665


## Inference
Such major swings in predictions in business will generally be hard to convince and may even be unacceptable. 